# 1. Web Scraping City information and transform into Dataframe
# 2. Separating into two Dataframes
# 4. Pushing to SQL Workbench

In [ ]:
#Libraries I used
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse
from sqlalchemy import create_engine 
import mysql.connector

## 1. Web Scraping - Manual Approch

### Berlin

In [ ]:
url = "https://en.wikipedia.org/wiki/Berlin" #the URL of the webpage I am getting the info from

response = requests.get(url)

soup_berlin = BeautifulSoup(response.content, 'html.parser') #extracting the raw html text

country_berlin = soup_berlin.find('td', class_='infobox-data').get_text(strip=True) #find the table and strip the text

lng_berlin = soup_berlin.find('span', class_='longitude').get_text(strip=True)
ltd_berlin = soup_berlin.find('span', class_='latitude').get_text(strip=True)

### Hamburg

In [3]:
url = "https://en.wikipedia.org/wiki/Hamburg"

response = requests.get(url)

soup_hamburg = BeautifulSoup(response.content, 'html.parser')

country_hamburg = soup_hamburg.find('td', class_='infobox-data').get_text(strip=True)

lng_hamburg = soup_hamburg.find('span', class_='longitude').get_text(strip=True)
ltd_hamburg = soup_hamburg.find('span', class_='latitude').get_text(strip=True)

### Munich

In [4]:
url = "https://en.wikipedia.org/wiki/Munich"

response = requests.get(url)

soup_munich = BeautifulSoup(response.content, 'html.parser')

country_munich = soup_munich.find('td', class_='infobox-data').get_text(strip=True)

lng_munich = soup_munich.find('span', class_='longitude').get_text(strip=True)
ltd_munich = soup_munich.find('span', class_='latitude').get_text(strip=True)

## 1. Web Scraping: Function to replace the manual steps above

In [5]:
#Function to replace steps above and to be able to use it for any other city
def cities_dataframe(cities):
  cities_data = []

  for city in cities:
    city_data = {}

    # city
    city_data["City"] = city

    # create the soup
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # country
    city_data["Country"] = city_soup.find(class_="infobox-data").get_text()

    # population
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = int(city_population.replace(",", ""))
    city_data["Population"] = city_population_clean

    # data retrieved
    city_data["Year_Data_Retrieved"] = city_soup.find(string="Population").find_next().get_text()[2:6]

    # latitude and longitude
    city_data["Latitude"] = city_soup.find(class_="latitude").get_text()
    city_data["Longitude"] = city_soup.find(class_="longitude").get_text()
    city_data["Latitude"] = parse(city_data["Latitude"])
    city_data["Longitude"] = parse(city_data["Longitude"])

    # append this city's data to the cities list
    cities_data.append(city_data)

  return pd.DataFrame(cities_data)

### Call the function with Cities names

In [6]:
# call the function
list_of_cities = ["Berlin", "Hamburg", "Munich"]

cities_df = cities_dataframe(list_of_cities)
cities_df

,City,Country,Population,Year_Data_Retrieved,Latitude,Longitude
0,Berlin,Germany,3596999,2022,52.5200,13.405
1,Hamburg,Germany,1964021,2023,53.5500,10.000
2,Munich,Germany,1510378,2023,48.1375,11.575


## 2. Seperating DATAFRAME into two tables

In [7]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 3 non-null      object 
 1   Country              3 non-null      object 
 2   Population           3 non-null      int64  
 3   Year_Data_Retrieved  3 non-null      object 
 4   Latitude             3 non-null      float64
 5   Longitude            3 non-null      float64
dtypes: float64(2), int64(1), object(3)
memory usage: 276.0+ bytes


In [8]:
# Create the "cities df" for the relational data base

cities_to_db = cities_df[["City", "Country", "Latitude", "Longitude"]]
cities_to_db

,City,Country,Latitude,Longitude
0,Berlin,Germany,52.5200,13.405
1,Hamburg,Germany,53.5500,10.000
2,Munich,Germany,48.1375,11.575


In [9]:
# Create the "population df" for the relational data base

population_to_db = cities_df[["Population", "Year_Data_Retrieved"]]
population_to_db

,Population,Year_Data_Retrieved
0,3596999,2022
1,1964021,2023
2,1510378,2023


In [10]:
#add airport codes
airport_codes = ['BER', 'HAM', 'MUC']
cities_to_db['Airport_Iata_Code']= airport_codes
cities_to_db

/var/folders/cn/bdqc214d4dqcr16tz7kb403w0000gn/T/ipykernel_5492/286484677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities_to_db['Airport_Iata_Code']= airport_codes


,City,Country,Latitude,Longitude,Airport_Iata_Code
0,Berlin,Germany,52.5200,13.405,BER
1,Hamburg,Germany,53.5500,10.000,HAM
2,Munich,Germany,48.1375,11.575,MUC


# 3. Pushing it into MySql Workbench

In the first step of this project, I pushed the info locally to mySQL Workbench and in the second step I pushed it into a database created on Google Cloud Provider (GCP). The steps are the same and only schema, host and password differs. I am sharing here only the second approch to push it directly to the cloud. 

In [ ]:
#Info for connection between mySQL Workbench and VS code
schema = "gans_cloud"
host = "PUBLIC IP" #host from Google cloud account (public IP)
user = "root"
password = password_GCP
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [14]:
# Push the "cities_to_db" to the empty "cities" table in the MySQL data base

cities_to_db.to_sql('Cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

### before pushing the populations DB into the cloud, reading back the Cities table to retrieve the foreign key (Cities_id)

In [16]:
cities_from_sql = pd.read_sql("Cities", con=connection_string) #reading back from sql
cities_from_sql

,City_id,City,Country,Longitude,Latitude,Airport_Iata_code
0,1,Berlin,Germany,13.405,52.5200,BER
1,2,Hamburg,Germany,10.000,53.5500,HAM
2,3,Munich,Germany,11.575,48.1375,MUC


In [17]:
# Getting the "City_id" to the population df--> needed for relational database connections

population_to_db["City_id"] = cities_from_sql["City_id"]
population_to_db

/var/folders/cn/bdqc214d4dqcr16tz7kb403w0000gn/T/ipykernel_5492/2823822354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_to_db["City_id"] = cities_from_sql["City_id"]


,Population,Year_Data_Retrieved,City_id
0,3596999,2022,1
1,1964021,2023,2
2,1510378,2023,3


In [18]:
# Push the "population_to_db" to the empty "population" table in the MySQL data base

population_to_db.to_sql('Population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3